In [1]:
!python -m pip install --upgrade pip


In [2]:
!pip install selenium
!pip install beautifulsoup4
!pip install webdriver-manager
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

In [10]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import os

# Ruta del archivo CSV
file_path = r'C:\Users\cande\Documents\Web Scraping\URLS_Null.csv'

# Verificar si el archivo existe
if not os.path.exists(file_path):
    print(f"El archivo {file_path} no existe.")
else:
    # Leer el archivo CSV con las URLs
    urls_df = pd.read_csv(file_path)

    # Verificar si la columna 'URL' existe en el DataFrame
    if 'URL' not in urls_df.columns:
        print("El archivo CSV no contiene una columna llamada 'URL'.")
    else:
        # Inicializar listas para almacenar los resultados
        ratings = []
        counts = []

        # Configurar el driver de Chrome
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

        # Iterar sobre cada URL en el DataFrame
        for url in urls_df['URL']:
            try:
                # Abrir la página de la aplicación
                driver.get(url)

                # Esperar un poco para que la página cargue completamente
                driver.implicitly_wait(10)

                # Obtener el contenido de la página
                html = driver.page_source

                # Analizar el contenido con BeautifulSoup
                soup = BeautifulSoup(html, 'html.parser')

                # Extraer el texto del elemento <figcaption>
                figcaption_element = soup.find('figcaption')
                if figcaption_element:
                    # Extraer el contenido del <figcaption>
                    content = figcaption_element.get_text(strip=True)
                    
                    # Separar el contenido en rating y count
                    if '•' in content:
                        rating, count = content.split('•')
                        rating = rating.strip()
                        count = count.strip().replace('Ratings', '').strip()
                    else:
                        rating = 'Formato no esperado'
                        count = 'Formato no esperado'
                else:
                    rating = 'Elemento no encontrado'
                    count = 'Elemento no encontrado'
            except Exception as e:
                rating = 'Error al procesar'
                count = str(e)

            # Agregar los resultados a las listas
            ratings.append(rating)
            counts.append(count)

        # Cerrar el navegador
        driver.quit()

        # Crear un DataFrame con los resultados
        result_df = pd.DataFrame({
            'URL': urls_df['URL'],
            'Average User Rating': ratings,
            'User Rating Count': counts
        })

        # Guardar los resultados en un archivo CSV
        result_df.to_csv(r'C:\Users\cande\Documents\Web Scraping\resultados.csv', index=False)

        print("Proceso completado. Los resultados se han guardado en 'resultados.csv'.")


Proceso completado. Los resultados se han guardado en 'resultados.csv'.


In [33]:
import pandas as pd

# Cargar el archivo principal
df_principal = pd.read_excel('archivo_principal.xlsx')

# Cargar el archivo con los datos restantes
df_complementario = pd.read_excel('archivo_complementario.xlsx')


In [38]:
# Combinar los DataFrames usando 'app URL' como clave
df_combinado = pd.merge(df_principal, df_complementario, on='App URL', how='left', suffixes=('', '_y'))

# Rellenar los valores nulos en 'Average User' y 'User count' con los datos del DataFrame complementario
df_combinado['Average User Rating'] = df_combinado['Average User Rating'].fillna(df_combinado['Average User Rating_y'])
df_combinado['User Rating Count'] = df_combinado['User Rating Count'].fillna(df_combinado['User Rating Count_y'])

# Eliminar las columnas adicionales que fueron creadas durante el merge
df_combinado = df_combinado.drop(columns=['Average User Rating_y', 'User Rating Count_y'])



In [44]:
import pandas as pd

# Cargar los archivos Excel en DataFrames
df_principal = pd.read_excel('archivo_principal.xlsx')
df_complementario = pd.read_excel('archivo_complementario.xlsx')

# Combinar los DataFrames usando 'App URL' como clave
df_combinado = pd.merge(df_principal, df_complementario, on='App URL', how='left', suffixes=('', '_y'))

# Rellenar los valores nulos en 'Average User Rating' y 'User Rating Count' con los datos del DataFrame complementario
df_combinado['Average User Rating'] = df_combinado['Average User Rating'].fillna(df_combinado['Average User Rating_y'])
df_combinado['User Rating Count'] = df_combinado['User Rating Count'].fillna(df_combinado['User Rating Count_y'])

# Crear un nuevo DataFrame solo con las columnas necesarias
df_final = df_combinado[['App URL', 'Average User Rating', 'User Rating Count']]

# Guardar el nuevo DataFrame en un archivo Excel
df_final.to_excel('archivo_combinado.xlsx', index=False)
